In [1]:
import tensorflow as tf
from tensorflow.layers import dense
import gym
import numpy as np

In [2]:
def make_cart_pole():
    return gym.make("CartPole-v1")

def discount_rewards(rewards, gamma=0.99):
    new_rewards = [rewards[-1]]
    for i in reversed(range(len(rewards)-1)):
        new_rewards.append(rewards[i] + gamma * new_rewards[-1])
    return new_rewards[::-1]

class MemoryBuffer():
    def __init__(self, max_size):
        self.max_size = max_size
        self.rollouts = []
        self.rollout_idx = -1
    
    def start_rollout(self):
        self.rollout_idx = (self.rollout_idx + 1) % self.max_size
        if self.rollout_idx >= len(self.rollouts):
            self.rollouts.append([])
        else:
            self.rollouts[self.rollout_idx] = []
            
    def end_rollout(self):
        self.start_rollout()
    
    def record(self, obs, act, rew):
        self.rollouts[self.rollout_idx].append([obs, act, rew])
        
    def to_data(self, reset=True):
        all_data = []
        
        try:
            for rollout in self.rollouts:
                rollout = np.array(rollout)
                # Discount the rewards for every rollout
                rollout[:,2] = discount_rewards(rollout[:,2])
                all_data.extend(list(rollout))

            if reset:
                self.reset()
        except IndexError:
            return np.array([])
            
        return np.array(all_data)
                
    def reset(self):
        self.rollouts = []
        self.rollout_idx = -1

In [11]:
class Network():
    def __init__(self, obs_shape, act_space, sess=None):
        self.obs_shape = obs_shape
        self.act_space = act_space
        self.sess = sess
        
        self.build_network()
        
    def build_network(self):
        self.obs = tf.placeholder(tf.float64, shape=[None, self.obs_shape])
        self.dense1 = dense(self.obs, 128, activation=tf.tanh)
        self.dense2 = dense(self.dense1, 128, activation=tf.tanh)
        self.act_probs = dense(self.dense2, self.act_space)
        self.softmax_probs = tf.nn.softmax(self.act_probs)
        self.chosen_act = tf.squeeze(tf.multinomial(logits=self.softmax_probs, num_samples=1), axis=1)
        self.actions = tf.placeholder(tf.int64, shape=[None])
        self.rewards = tf.placeholder(tf.float64, shape=[None])
        
        self.action_masks = tf.one_hot(self.actions, self.act_space, dtype=tf.float64)
        self.log_probs = tf.log(self.softmax_probs)
        
        self.resp_actions = tf.reduce_sum(self.action_masks *  self.log_probs, axis=1)
        self.loss = -tf.reduce_mean(self.resp_actions * self.rewards)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.update = self.optimizer.minimize(self.loss)
        
#         self.action_masks = tf.one_hot(self.actions, self.act_space, dtype=tf.float64)
#         self.resp_actions = tf.reduce_sum(self.action_masks * tf.log(self.out), axis=1)
        
#         self.loss = -tf.reduce_mean(self.resp_actions * self.rewards)
        
# #         self.loss = -tf.reduce_mean(tf.transpose(tf.log(self.resp_actions)) * self.rewards)
#         self.optimizer = tf.train.AdamOptimizer()
#         self.optimize = self.optimizer.minimize(self.loss)
        
    def init_session(self):
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.sess.__enter__()
        return self.sess
    
    def end_session(self):
        if self.sess is not None:
            self.sess.close()
        
    def choose_action1(self, obs, sess=None):
        if not sess:
            if self.sess is not None:
                sess = self.sess
            else:
                sess = tf.get_default_session()
                assert sess is not None
        
        act = sess.run(self.chosen_act, feed_dict={self.obs: obs})
        return act
    
    def choose_action2(self, obs, sess=None):
        if not sess:
            if self.sess is not None:
                sess = self.sess
            else:
                sess = tf.get_default_session()
                assert sess is not None
        
        probs = sess.run(self.softmax_probs, feed_dict={self.obs: obs})
        act = np.random.choice(list(range(len(probs)+1)), p=probs[0])
        
        return act, probs
    
    def train(self, train_data, sess=None):
        """train_data: in the two-dimensional numpy array with the format, [obs, act, rew]"""
        if not sess:
            if self.sess is not None:
                sess = self.sess
            else:
                sess = tf.get_default_session()
                assert sess is not None
        
        a, b, _ = sess.run([self.softmax_probs, tf.multinomial(logits=self.softmax_probs,num_samples=1), self.update], feed_dict={self.obs: np.vstack(train_data[:,0]),
                                            self.actions: train_data[:,1],
                                            self.rewards: train_data[:,2]})
#         print(a[:50])
#         print(b[:50])

In [12]:
n_episodes = 10000
max_steps = 200
update_freq = 200 # In episodes

In [13]:
mb = MemoryBuffer(update_freq)
env = make_cart_pole()
network = Network(env.observation_space.shape[0], env.action_space.n)
network.init_session();

In [14]:
import time
total_steps = 0
total_episodes = 0
all_rewards = []

for episode in range(n_episodes):
    obs = env.reset()
    
    episode_reward = 0
    mb.start_rollout()
    for step in range(max_steps):
        act, _ = network.choose_action2([obs])
        act2 = network.choose_action1([obs])[0]
        
        if total_steps % 500 == 0:
            print(_, act, act2)

        # print(act)
        # env.render()
        # time.sleep(0.01)
        obs_next, rew, d, _ = env.step(act)
        episode_reward += rew
        
        mb.record(obs, act, rew)
        obs = obs_next
        
        total_steps += 1
        if d: # or total_steps % update_freq == 0:
            #  print(act)
            break
    # print(episode_reward)
    all_rewards.append(episode_reward)
    total_episodes += 1
            
    if total_episodes % 100 == 0:
        print('Recent Reward:', np.mean(all_rewards[-100:]))
        print('Total Episodes:', total_episodes)
        print('Total Steps:', total_steps)
        print('\n-----------')
        
        train_data = mb.to_data()
#         rews = train_data[:,2]
#         train_data[:,2] = (rews - np.mean(rews)) / np.std(rews)
        network.train(train_data)

[[0.50038762 0.49961238]] 1 1
[[0.52890111 0.47109889]] 1 0
[[0.54616041 0.45383959]] 1 0
[[0.50775307 0.49224693]] 0 1
Recent Reward: 19.78
Total Episodes: 100
Total Steps: 1978

-----------
[[0.48490568 0.51509432]] 1 1
[[0.48429321 0.51570679]] 1 1
[[0.48589296 0.51410704]] 0 1
[[0.49357878 0.50642122]] 1 0
[[0.48060973 0.51939027]] 0 0
Recent Reward: 22.74
Total Episodes: 200
Total Steps: 4252

-----------
[[0.65127712 0.34872288]] 0 0
[[0.60214459 0.39785541]] 0 1
[[0.50874221 0.49125779]] 0 0
[[0.52476519 0.47523481]] 0 0
[[0.36807044 0.63192956]] 0 0
Recent Reward: 27.23
Total Episodes: 300
Total Steps: 6975

-----------
[[0.60068386 0.39931614]] 0 0
[[0.55868244 0.44131756]] 0 0
[[0.4882034 0.5117966]] 1 0
[[0.25701717 0.74298283]] 1 1
[[0.44029533 0.55970467]] 0 0
[[0.45022957 0.54977043]] 1 1
[[0.56965072 0.43034928]] 1 1
Recent Reward: 34.58
Total Episodes: 400
Total Steps: 10433

-----------
[[0.46080438 0.53919562]] 1 0
[[0.49342774 0.50657226]] 0 1
[[0.58354318 0.41645682

[[0.69062745 0.30937255]] 0 0
[[0.82523708 0.17476292]] 0 0
[[0.50934616 0.49065384]] 1 0
[[0.64818268 0.35181732]] 0 1
[[0.629458 0.370542]] 1 1
[[0.3469152 0.6530848]] 1 0
[[0.96352979 0.03647021]] 0 0
[[0.39988549 0.60011451]] 1 1
[[0.7137062 0.2862938]] 0 0
[[0.39190754 0.60809246]] 1 1
[[0.43707486 0.56292514]] 1 0
[[0.20588646 0.79411354]] 0 0
[[0.27759081 0.72240919]] 1 0
[[0.74197727 0.25802273]] 0 1
[[0.11352292 0.88647708]] 1 1
[[0.73603044 0.26396956]] 0 1
[[0.73703733 0.26296267]] 0 1
[[0.77455246 0.22544754]] 0 0
[[0.39698152 0.60301848]] 1 0
[[0.67512671 0.32487329]] 0 1
[[0.47255253 0.52744747]] 1 0
[[0.09307498 0.90692502]] 1 1
[[0.68296504 0.31703496]] 1 1
[[0.79970019 0.20029981]] 0 0
[[0.14418392 0.85581608]] 1 0
[[0.35664519 0.64335481]] 1 1
[[0.26607897 0.73392103]] 1 1
[[0.25022528 0.74977472]] 0 0
[[0.14957508 0.85042492]] 1 0
Recent Reward: 146.37
Total Episodes: 1900
Total Steps: 129977

-----------
[[0.57925102 0.42074898]] 0 0
[[0.89818542 0.10181458]] 0 0
[[

KeyboardInterrupt: 

In [ ]:
network.end_session()

In [ ]:
mb.to_data().shape